# Context-aware Maritime Movement Analysis


This notebook uses the open science 

"Heterogeneous Integrated Dataset for Maritime Intelligence, Surveillance, and Reconnaissance" (https://zenodo.org/record/1167595)

by RAY, Cyril; DRÉO, Richard; CAMOSSI, Elena; JOUSSELME, Anne-Laure 

In [ ]:
import io
from os.path import exists
from urllib.request import urlretrieve
from zipfile import ZipFile
from datetime import timedelta

import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import hvplot.pandas 

import warnings
warnings.filterwarnings('ignore')

mpd.show_versions()

In [ ]:
def download_from_zenodo(zip_file, url):
    if not exists(zip_file):
        urlretrieve(url, zip_file)

The following cells only show a subset of the files included in https://zenodo.org/record/1167595. Visit the Zenodo page for a full overview

In [ ]:
ais_zip_file = 'data/P1_AIS_Data.zip'
download_from_zenodo(ais_zip_file, 'https://zenodo.org/record/1167595/files/%5BP1%5D%20AIS%20Data.zip?download=1')
ports_zip_file = 'data/C1_Ports_Brittany.zip'
download_from_zenodo(ports_zip_file, 'https://zenodo.org/record/1167595/files/%5BC1%5D%20Ports%20of%20Brittany.zip?download=1')
fishing_areas_zip_file = 'data/C4_Fishing_areas.zip'
download_from_zenodo(fishing_areas_zip_file, 'https://zenodo.org/record/1167595/files/%5BC4%5D%20Fishing%20Areas%20%28European%20commission%29.zip?download=1')

In [ ]:
ports_gdf = gpd.read_file(ports_zip_file)
fish_areas_gdf = gpd.read_file(fishing_areas_zip_file)
ports_gdf.hvplot(title='Ports of Brittany', geo=True, tiles='OSM', hover_cols=['libelle_po']) * fish_areas_gdf.hvplot(geo=True, c='value', line_width=0, cmap='Oranges', alpha=0.5)

In [ ]:
MIN_TRAJ_LENGTH = 1000 # m
RESAMPLING_INTERVAL = timedelta(minutes=1)
NROWS = 10000

def csv_to_traj_collection(csv_file_name):
    with ZipFile(ais_zip_file) as z:
        with z.open(csv_file_name) as f:
            df = pd.read_csv(f, nrows=NROWS)
            #print(df.head())
            try:
                df['t'] = pd.to_datetime(df['ts'],unit='s')
                df.drop(columns=['ts'])
            except:
                df['t'] = pd.to_datetime(df['t'],unit='s')
            df = df[['t', 'sourcemmsi', 'lon', 'lat']]
            tc = mpd.TrajectoryCollection(df, 'sourcemmsi', x='lon', y='lat', t='t', min_length=MIN_TRAJ_LENGTH)
            tc = mpd.MinTimeDeltaGeneralizer(tc).generalize(tolerance=RESAMPLING_INTERVAL)
            return tc

In [ ]:
sar_tc = csv_to_traj_collection("nari_dynamic_sar.csv")
sar_tc

In [ ]:
sar_tc.hvplot(title="SAR Trajectories", line_width=5, hover_cols=['sourcemmsi'])

In [ ]:
tc = csv_to_traj_collection("nari_dynamic.csv")
tc

In [ ]:
fising_area_plot = fish_areas_gdf.hvplot(geo=True, c='value', line_width=0, cmap='Oranges', alpha=0.5, tiles='OSM')
port_plot = ports_gdf.hvplot(geo=True, hover_cols=['libelle_po'], color='black')
ais_plot = tc.hvplot(title='AIS sample, ports, and fishing areas', line_width=5, hover_cols=['sourcemmsi'], tiles=None) 
fising_area_plot * port_plot * ais_plot